In [1]:
import pandas as pd
import numpy as np
import redis
import py2neo
from pprint import pprint

In [2]:
# Connectiong to the Redis DB:
r = redis.Redis(
    host='rhea.isegi.unl.pt',                            # Url where the database is hosted
    port='6404',                                         # Server port in which redis is running
    password='249774l78m84D1507a9GY9410sy08U83', # Password to authenticate on the server
    decode_responses=True
)

server_info = r.info()

print("Server version "+ server_info["redis_version"])

Server version 6.2.2


In [3]:
# Connecting to the neo4j DB
username="neo4j"
password="F3cfcrnvBev57KZ8mcMk78L9wHgJVZuJ"
host="rhea.isegi.unl.pt"
port="7474"

secure_graph = py2neo.Graph(f"http://{username}:{password}@{host}:{port}")
secure_graph.run("MATCH () RETURN count(*)").data()

[{'count(*)': 9647598}]

Authors: 

* Maikel Sousa (m20200735)
* Catarina Moreira (20201034)
* Luisa Barral (m20201045)
* Xavier Goncalves (m20201090)


# Answering the questions:

#### 0) How many beers does the database contain?

In [18]:
result = secure_graph.run("""
        MATCH 
            (n:Beers)
        RETURN 
            count(distinct n) AS Distinct
""").data()

print(f"We have {result[0]['Distinct']} Beers in the Neo4j Database")

We have 358873 Beers in the Neo4j Database


In [19]:
# Uploading the result to the redis database:
r.set("0", result[0]['Distinct'])

True

#### 1) How many different countries exist in the database?

In [24]:
result = secure_graph.run("""
        MATCH 
            (n:Country)
        RETURN 
            count(distinct n) AS Distinct
""").data()

print(f"We have {result[0]['Distinct']} Countries in the Neo4j Database")

We have 200 Countries in the Neo4j Database


In [25]:
# Uploading the result to the redis database:
r.set("1", result[0]['Distinct'])

True

### 2) Most Reviews:

#### 2A) Which Beer has the most reviews?

In [26]:
result = secure_graph.run("""
        MATCH 
            (n:Beers)-[a:ABOUT]-(r:Reviews)
        WITH 
            n['name'] AS BeerName, 
            count(r) AS NumberReviews
        ORDER BY 
            NumberReviews DESC
        RETURN 
            BeerName,
            NumberReviews
        LIMIT 4
""").data()

print(f"The beer with most reviews is {result[0]['BeerName']} with {result[0]['NumberReviews']} reviews")

The beer with most reviews is IPA with 31387 reviews


In [27]:
# Uploading the result to the redis database:
r.set("2A", result[0]['BeerName'])

True

#### 2.2) Which Brewery has the most reviews for its beers?

In [28]:
result = secure_graph.run("""
        MATCH 
            (brew:Breweries)-[ba:BREWED_AT]-(b:Beers)-[a:ABOUT]-(r:Reviews)
        WITH
            brew['name'] AS Brewery,
            count(r) AS NumberReviews
        ORDER BY
            NumberReviews DESC
        RETURN 
            Brewery,
            NumberReviews
        LIMIT 4
""").data()

print(f"The Brewery with most reviews is {result[0]['Brewery']} with {result[0]['NumberReviews']} reviews")

The Brewery with most reviews is Sierra Nevada Brewing Co. with 175161 reviews


In [29]:
# Uploading the result to the redis database:
r.set("2B", result[0]['Brewery'])

True

#### 2.3) Which Country has the most reviews for its beers?

In [30]:
result = secure_graph.run("""
        MATCH 
            (c:Country)<-[f:FROM]-(brew:Breweries)-[ba:BREWED_AT]-(b:Beers)-[a:ABOUT]-(r:Reviews)
        WITH
            c['country_digit'] AS CountryCode,
            count(r) AS NumberReviews
        ORDER BY
            NumberReviews DESC
        RETURN 
            CountryCode,
            NumberReviews
        LIMIT 4
""").data()

#pprint(result)
print(f"The Country with most reviews is {result[0]['CountryCode']} with {result[0]['NumberReviews']} reviews")

The Country with most reviews is US with 7524410 reviews


In [31]:
# Uploading the result to the redis database:
r.set("2C", result[0]['CountryCode'])

True

#### 3) Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [32]:
# Checking the query:
result = secure_graph.run("""
        MATCH 
            (b0:Beers)-[:ABOUT]-(r0:Reviews)-[:MADE]-(u0:Username)
        MATCH 
            (b:Beers)-[:ABOUT]-(r:Reviews)-[:MADE]-(u:Username{user_name:"CTJman"})
        
        WHERE
            b0['name'] = b['name']
        
        WITH
            u0['user_name'] AS User,
            count(r0) as N_Reviews
            
        ORDER BY
            N_Reviews DESC
        
        RETURN
            User,
            N_Reviews
            
        LIMIT 10
        
""").data()

pprint(result)

[{'N_Reviews': 2241, 'User': 'CTJman'},
 {'N_Reviews': 1789, 'User': 'acurtis'},
 {'N_Reviews': 1662, 'User': 'Texasfan549'},
 {'N_Reviews': 1532, 'User': 'kjkinsey'},
 {'N_Reviews': 1412, 'User': 'djrn2'},
 {'N_Reviews': 1394, 'User': 'spycow'},
 {'N_Reviews': 1377, 'User': 'oline73'},
 {'N_Reviews': 1368, 'User': 'jaydoc'},
 {'N_Reviews': 1339, 'User': 'Knapp85'},
 {'N_Reviews': 1327, 'User': 'kylehay2004'}]


In [33]:
print(f"The user with the most shared reviews with CTJman is: {result[0]['User']}")

The user with the most shared reviews with CTJman is: CTJman


In [34]:
# Uploading the result to the redis database:
r.set("3", result[0]['User'])

True

#### 4) Which portuguese brewery has the most beers?

In [35]:
result = secure_graph.run("""
        MATCH 
            (c:Country{country_digit:'PT'})<-[f:FROM]-(brew:Breweries)-[ba:BREWED_AT]-(b:Beers)
        WITH 
            brew['name'] AS Brewery,
            count(b) AS N_Beers
        ORDER BY
            N_Beers DESC
        RETURN
            Brewery
        LIMIT 4
""").data()

print(f"The Portuguese Brewery with most beers is: \'{ result[0]['Brewery'] }\' ")
#pprint(result)

The Portuguese Brewery with most beers is: 'Dois Corvos Cervejeira' 


In [36]:
# Uploading the result to the redis database:
r.set("4", result[0]['Brewery'])

True

#### 5) From those beers (the ones returned from the previous question), which has the most reviews?

In [37]:
result = secure_graph.run("""
        MATCH 
            (c:Country{country_digit:'PT'})<-[f:FROM]-(brew:Breweries{name:'Dois Corvos Cervejeira'})-[ba:BREWED_AT]-(b:Beers)-[a:ABOUT]-(r:Reviews)
        WITH 
            b['name'] AS Beer,
            count(r) AS N_Reviews
        ORDER BY
            N_Reviews DESC
        RETURN
            Beer,
            N_Reviews
        LIMIT 4
""").data()

print(f"The Portuguese Brewery Dois Corvos Cervejeira\'s beer with more reviews is: \'{ result[0]['Beer'] }\' ")
#pprint(result)

The Portuguese Brewery Dois Corvos Cervejeira's beer with more reviews is: 'Finisterra' 


In [38]:
# Uploading the result to the redis database:
r.set("5", result[0]['Beer'])

True

Just out of curiosity, we queried what is the most reviewed beer in Portugal:

In [39]:
result = secure_graph.run("""
        MATCH 
            (c:Country{country_digit:'PT'})<-[f:FROM]-(brew:Breweries)-[ba:BREWED_AT]-(b:Beers)-[a:ABOUT]-(r:Reviews)
        WITH 
            b['name'] AS Beer,
            count(r) AS N_Reviews
        ORDER BY
            N_Reviews DESC
        RETURN
            Beer,
            N_Reviews
        LIMIT 4
""").data()

print(f"The Portuguese Brewery with most reviews is: \'{ result[0]['Beer'] }\' ")
#pprint(result)

The Portuguese Brewery with most reviews is: 'Super Bock' 


#### 6) On average how many different beer styles does each brewery produce?

In [40]:
# Explore query:
result = secure_graph.run("""
        MATCH 
            (brew:Breweries)-[ba:BREWED_AT]-(b:Beers)-[t:OF_TYPE]-(s:Style)
        WITH
            brew,
            count(s) AS N_Styles
        ORDER BY
            N_Styles ASC
        RETURN
            brew["name"] AS Brewery,
            N_Styles
        LIMIT 10
""").data()

pprint(result)

[{'Brewery': 'Mallaskuun Panimo', 'N_Styles': 1},
 {'Brewery': 'The Cellar Grill', 'N_Styles': 1},
 {'Brewery': 'Microbrasserie Senses Brewing', 'N_Styles': 1},
 {'Brewery': 'Vechtdal Brouwerij', 'N_Styles': 1},
 {'Brewery': 'Real Cervejeira Caldense', 'N_Styles': 1},
 {'Brewery': 'Brasserie IRVOY', 'N_Styles': 1},
 {'Brewery': 'Gallia Omnia', 'N_Styles': 1},
 {'Brewery': 'Brasserie La Belette', 'N_Styles': 1},
 {'Brewery': 'Mozencahua Brewery', 'N_Styles': 1},
 {'Brewery': 'Hobbybrouwerij de Hopjutters', 'N_Styles': 1}]


At least each brewery has a Style of beer. Therefore, we take the average of the N_Styles.

In [41]:
# Final Query:
result = secure_graph.run("""
        MATCH 
            (brew:Breweries)-[ba:BREWED_AT]-(b:Beers)-[t:OF_TYPE]-(s:Style)
        WITH
            brew,
            count(s) AS N_Styles
        RETURN
            avg(N_Styles) AS Average
        LIMIT 4
""").data()

print(f"On average, a Brewery produces: {np.round(result[0]['Average'],2)}, different styles of beer")

On average, a Brewery produces: 21.66, different styles of beer


In [42]:
# Uploading the result to the redis database:
r.set("6", np.round(result[0]['Average'],2))

True

#### 7) Which brewery produces the strongest beers according to ABV?

In [43]:
result = secure_graph.run("""
        MATCH 
            (brew:Breweries)-[ba:BREWED_AT]-(b:Beers)
        WITH
            brew.name AS Brewery,
            b.name AS BeerName,
            toFloat(b.abv) AS ABV
        ORDER BY
            ABV DESC
        WHERE
            ABV >= 0 
        RETURN
            Brewery,
            BeerName,
            ABV
        LIMIT 7
""").data()

#print(f"The Brewery with the strongest beer acording to ABV is: \'{result[0]['Brewery']}\', with an ABV of {result[0]['ABV']} it\'s name is: {result[0]['BeerName']}")
pprint(result)

[{'ABV': 100.0, 'BeerName': 'water', 'Brewery': 'Avondale Brewing Co.'},
 {'ABV': 100.0,
  'BeerName': 'Radiohead - OK Computer',
  'Brewery': '1648 Brewing Company Ltd'},
 {'ABV': 100.0,
  'BeerName': "Earache: World's Shortest Album",
  'Brewery': '1648 Brewing Company Ltd'},
 {'ABV': 80.0,
  'BeerName': 'Dark Reckoning',
  'Brewery': 'Morgan Territory Brewing'},
 {'ABV': 67.5, 'BeerName': 'Snake Venom', 'Brewery': 'Brewmeister'},
 {'ABV': 66.0, 'BeerName': 'series 3', 'Brewery': 'Redline Brewhouse'},
 {'ABV': 65.0, 'BeerName': 'Armageddon', 'Brewery': 'Brewmeister'}]


Although we have weird looking values for the ABV of beers, like water, Radiohead or Dark Reckoning, which has in fact 8% according to their website.

Taking into consideration only the values that make sense, the beer with the highest ABV is Snake Venom, but, since it was clarified on teams, the answer to this question is on the query below:

In [44]:
result = secure_graph.run("""
        MATCH 
            (brew:Breweries)-[ba:BREWED_AT]-(b:Beers)
        WITH
            brew.name AS Brewery,
            b.name AS BeerName,
            toFloat(b.abv) AS ABV
        WHERE
            ABV >= 0
        RETURN
            Brewery,
            avg(ABV) AS AvgABV
        ORDER BY
            AvgABV DESC
        LIMIT 5
""").data()

print(f"The Brewery with the strongest beers by ABV is: \'{result[0]['Brewery']}\', with an average ABV of {np.round(result[0]['AvgABV'],2)}")
#pprint(result)

The Brewery with the strongest beers by ABV is: '1648 Brewing Company Ltd', with an average ABV of 25.58


In [45]:
# Uploading the result to the redis database:
r.set("7", result[0]['Brewery'] )

True

#### 8) If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [46]:
result = secure_graph.run("""
        MATCH 
            (s:Style)-[t:OF_TYPE]-(b:Beers)-[:ABOUT]-(r:Reviews)
        WHERE
            r.look <> "Unknown" and r.smell <> "Unknown"
        RETURN
            s.name AS Style,
            avg(toFloat(r.look)) AS AvgAppearance,
            avg(toFloat(r.smell)) AS AvgAroma
        ORDER BY
            AvgAroma DESC,
            AvgAppearance DESC
        LIMIT 5
""").data()

pprint(result)

[{'AvgAppearance': 4.383595613210904,
  'AvgAroma': 4.41361476476119,
  'Style': 'New England IPA'},
 {'AvgAppearance': 4.286392481437848,
  'AvgAroma': 4.2628841976746354,
  'Style': 'American Imperial Stout'},
 {'AvgAppearance': 4.1749641745318,
  'AvgAroma': 4.230876612146084,
  'Style': 'Belgian Gueuze'},
 {'AvgAppearance': 4.195264823940161,
  'AvgAroma': 4.1930865124172065,
  'Style': 'American Imperial Porter'},
 {'AvgAppearance': 4.014417114510136,
  'AvgAroma': 4.153730880529142,
  'Style': 'German Eisbock'}]


In [47]:
print(f"The recommended beer style for you is: \'{result[0]['Style']}\' ")

The recommended beer style for you is: 'New England IPA' 


In [48]:
# Uploading the result to the redis database:
r.set("8", result[0]['Style'] )

True

#### 9) Using Graph Algorithms answer two of the following questions:

    A. Which two Countries are most similiar when it comes to their **top 10** most produced Beer styles?
    
First let's create a graph in memory, for that, let's see if each of the nodes and relationships are well defined...

Nodes:

In [49]:
# TOP 10 styles by country.
# (c:Country)<-[:FROM]-(brew:Breweries)-[ba:BREWED_AT]-(b:Beers)-[t:OF_TYPE]-(s:Style)
secure_graph.run("""
    MATCH (c:Country) return id(c) as id 
    UNION ALL  
    MATCH (s:Style) return id(s) as id
    LIMIT 10
""").data()

[{'id': 59848},
 {'id': 59850},
 {'id': 59854},
 {'id': 59857},
 {'id': 59859},
 {'id': 59866},
 {'id': 59869},
 {'id': 59871},
 {'id': 59877},
 {'id': 59879},
 {'id': 59888},
 {'id': 59896},
 {'id': 59910},
 {'id': 59920},
 {'id': 59932},
 {'id': 59935},
 {'id': 59944},
 {'id': 59947},
 {'id': 59949},
 {'id': 59994},
 {'id': 60018},
 {'id': 60043},
 {'id': 60049},
 {'id': 60051},
 {'id': 60055},
 {'id': 60059},
 {'id': 60082},
 {'id': 60132},
 {'id': 60160},
 {'id': 60186},
 {'id': 60197},
 {'id': 60229},
 {'id': 60234},
 {'id': 60240},
 {'id': 60293},
 {'id': 60319},
 {'id': 60349},
 {'id': 60368},
 {'id': 60373},
 {'id': 60410},
 {'id': 60416},
 {'id': 60423},
 {'id': 60445},
 {'id': 60451},
 {'id': 60505},
 {'id': 60507},
 {'id': 60522},
 {'id': 60542},
 {'id': 60547},
 {'id': 60582},
 {'id': 60588},
 {'id': 60621},
 {'id': 60630},
 {'id': 60640},
 {'id': 60689},
 {'id': 60712},
 {'id': 60740},
 {'id': 60761},
 {'id': 60800},
 {'id': 60806},
 {'id': 60813},
 {'id': 60890},
 {'id': 

Now, let's create the relationships between the nodes:

In [123]:
# TOP 10 styles by country.
secure_graph.run("""
    MATCH (c:Country)<-[]-()-[]-()-[]-(s:Style)
    WITH c AS Country, s AS Styles, count(*) AS times
    ORDER BY times DESC
    WITH Country, collect(Styles)[..10] as top_10
    UNWIND top_10 AS s_top_10
    RETURN id(Country) AS source, id(s_top_10) AS target
    LIMIT 10
""").data()

[{'source': 59850, 'target': 110409},
 {'source': 59850, 'target': 110426},
 {'source': 59850, 'target': 110444},
 {'source': 59850, 'target': 110457},
 {'source': 59850, 'target': 110446},
 {'source': 59850, 'target': 110401},
 {'source': 59850, 'target': 110439},
 {'source': 59850, 'target': 110470},
 {'source': 59850, 'target': 110479},
 {'source': 59850, 'target': 110539}]

Visualizing the query above in a understandable way:

In [213]:
# TOP 10 styles by country.
secure_graph.run("""
    MATCH (c:Country)<-[]-()-[]-()-[]-(s:Style)
    WITH c AS Country, s AS Styles, count(*) AS times
    ORDER BY times DESC
    WITH Country, collect(Styles)[..10] as top_10
    UNWIND top_10 AS s_top_10
    RETURN Country.country_digit AS source, s_top_10.name AS target
    LIMIT 20
""").data()

[{'source': 'US', 'target': 'American IPA'},
 {'source': 'US', 'target': 'American Pale Ale (APA)'},
 {'source': 'US', 'target': 'American Imperial IPA'},
 {'source': 'US', 'target': 'Belgian Saison'},
 {'source': 'US', 'target': 'American Wild Ale'},
 {'source': 'US', 'target': 'American Imperial Stout'},
 {'source': 'US', 'target': 'American Porter'},
 {'source': 'US', 'target': 'American Stout'},
 {'source': 'US', 'target': 'American Amber / Red Ale'},
 {'source': 'US', 'target': 'American Brown Ale'},
 {'source': 'CA', 'target': 'American IPA'},
 {'source': 'CA', 'target': 'American Pale Ale (APA)'},
 {'source': 'CA', 'target': 'Belgian Saison'},
 {'source': 'CA', 'target': 'American Wild Ale'},
 {'source': 'CA', 'target': 'Fruit and Field Beer'},
 {'source': 'CA', 'target': 'American Imperial IPA'},
 {'source': 'CA', 'target': 'American Amber / Red Ale'},
 {'source': 'CA', 'target': 'American Blonde Ale'},
 {'source': 'CA', 'target': 'American Porter'},
 {'source': 'CA', 'target':

In [222]:
# Creating in memory graph
# (c:Country)<-[:FROM]-(brew:Breweries)-[ba:BREWED_AT]-(b:Beers)-[t:OF_TYPE]-(s:Style)

secure_graph.run("""                                    
    CALL gds.graph.create.cypher(
        'my-graph-group14_final',
        'MATCH (c:Country) return id(c) as id
        UNION ALL
        MATCH (s:Style) return id(s) as id',
        'MATCH 
            (c:Country)<-[]-()-[]-()-[]-(s:Style)
        WITH 
            c AS Country, s AS Styles, count(*) AS times
        ORDER BY 
            times DESC
        WITH 
            Country, collect(Styles)[..10] as top_10
        UNWIND 
            top_10 AS s_top_10
        RETURN 
            id(Country) AS source, 
            id(s_top_10) AS target'
    )
""").data()

[{'nodeQuery': 'MATCH (c:Country) return id(c) as id\n        UNION ALL\n        MATCH (s:Style) return id(s) as id',
  'relationshipQuery': 'MATCH \n            (c:Country)<-[]-()-[]-()-[]-(s:Style)\n        WITH \n            c AS Country, s AS Styles, count(*) AS times\n        ORDER BY \n            times DESC\n        WITH \n            Country, collect(Styles)[..10] as top_10\n        UNWIND \n            top_10 AS s_top_10\n        RETURN \n            id(Country) AS source, \n            id(s_top_10) AS target',
  'graphName': 'my-graph-group14_final',
  'nodeCount': 313,
  'relationshipCount': 1365,
  'createMillis': 8047}]

Finally, let's get the similarity metrics between the countries that look more alike in their top 10 countries:

In [50]:
result = secure_graph.run("""     
CALL gds.nodeSimilarity.stream('my-graph-group14_final')
YIELD node1, node2, similarity
RETURN gds.util.asNode(node1).country_digit AS Country1, gds.util.asNode(node2).country_digit AS Country2, similarity
ORDER BY similarity DESCENDING
LIMIT 10
""").data()

pprint(result)

[{'Country1': 'TD', 'Country2': 'TJ', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'CF', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'ST', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'YT', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'BF', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'AO', 'similarity': 1.0},
 {'Country1': 'IR', 'Country2': 'LY', 'similarity': 1.0},
 {'Country1': 'IR', 'Country2': 'BW', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'SN', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'MO', 'similarity': 1.0}]


In [51]:
print(f"The countries with that look the most alike based on their top 10 beer styles are {result[0]['Country1']} and {result[0]['Country2']}")

The countries with that look the most alike based on their top 10 beer styles are TD and TJ


In [53]:
# Uploading the result to the redis database:
r.set("9A", result[0]['Country1'] + ' and ' + result[0]['Country2'] )

True

#### 9.C) Which user is the most influential when it comes to reviews made?

To find influence within the users in the database we will apply the page rank algorithm to the users and reviews nodes.

In [55]:
data = secure_graph.run(
    """
        CALL gds.graph.create(
            '9c-group14-my-graph',
            [
                'Username',
                'Reviews'
            ],
            {
                MADE: {
                    orientation: 'REVERSE'
                }
            }
        )
    """
).data()

In [56]:
data = secure_graph.run(
    """
        CALL gds.pageRank.stream('9c-group14-my-graph') 
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).user_name AS username, score
        ORDER BY score DESC 
        LIMIT 10
    """
).data()
data

[{'username': 'Sammy', 'score': 1759.3714309692384},
 {'username': 'kylehay2004', 'score': 1558.297300720215},
 {'username': 'acurtis', 'score': 1532.1618240356445},
 {'username': 'StonedTrippin', 'score': 1512.1404693603515},
 {'username': 'jaydoc', 'score': 1504.620169067383},
 {'username': 'UCLABrewN84', 'score': 1433.0861373901369},
 {'username': 'Texasfan549', 'score': 1425.9496963500976},
 {'username': 'kjkinsey', 'score': 1307.375163269043},
 {'username': 'GRG1313', 'score': 1277.7963668823243},
 {'username': 'Knapp85', 'score': 1249.6200134277344}]

In [57]:
print(f"The most influential user is: \'{data[0]['username']}\' ")

The most influential user is: 'Sammy' 


In [58]:
# Uploading the result to the redis database:
r.set("9C", data[0]['username'] )

True

#### 10) If you had to pick 3 beers to recommend using only this database, which would you pick and why?

We will assess the beers to recommend based on the total number of reviews, higher than average overall score and higher than average taste score. Being the average overall score when is over 4.

In [59]:
result = secure_graph.run("""
        MATCH 
            (b:Beers)-[:ABOUT]-(r:Reviews)
        WHERE
            r.taste <> "Unknown" and r.overall <> "Unknown"
        WITH 
            b.name AS BeerName,
            count(r) AS NumberReviews,
            avg(toFloat(r.overall)) AS AvgOverall,
            avg(toFloat(r.taste)) AS AvgTaste
        WHERE
            AvgOverall > 4
        RETURN 
            BeerName,
            NumberReviews,
            AvgOverall,
            AvgTaste
        ORDER BY
            NumberReviews DESC,
            AvgOverall DESC,
            AvgTaste DESC
        LIMIT 3
""").data()

result

[{'BeerName': 'Imperial Stout',
  'NumberReviews': 10176,
  'AvgOverall': 4.1334512578616245,
  'AvgTaste': 4.1940104166666865},
 {'BeerName': 'Breakfast Stout',
  'NumberReviews': 7663,
  'AvgOverall': 4.435958501892222,
  'AvgTaste': 4.491060942189744},
 {'BeerName': 'KBS (Kentucky Breakfast Stout)',
  'NumberReviews': 7042,
  'AvgOverall': 4.526732462368638,
  'AvgTaste': 4.592125816529386}]

In [60]:
print(f"The 3 beers we recommend are:\n{[ result[i]['BeerName'] for i in range(len(result)) ]}")

The 3 beers we recommend are:
['Imperial Stout', 'Breakfast Stout', 'KBS (Kentucky Breakfast Stout)']


In [61]:
# Uploading the result to the redis database:
r.set("10", result[0]['BeerName'] + ' - ' + result[1]['BeerName'] + ' - ' + result[2]['BeerName'] )

True